In [6]:
import csv
import random
from faker import Faker
from datetime import timedelta

### CSV EMPLEADOS

In [7]:
fake = Faker('es_ES')

In [8]:
num_empleados = 150
roles = ['Dependiente', 'Jefe de sección', 'Jefe de tienda', 'Director de tienda', 'Gerente de tienda', 'Subdirector de tienda', 'Encargado de tienda']
opciones_tf = ['Sí', 'No']

with open('../data/empleados.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(['id_empleado', 'nombre_completo', 'edad_empleado', 'antiguedad_anos', 'personas_a_cargo', 'rol_empleado', 'minusvalia', 'activo', 'id_departamento'])

    for i in range(num_empleados):
        nombre = fake.first_name()
        apellidos = fake.last_name() + ' ' + fake.last_name()
        edad = random.randint(18, 65)
        antiguedad_maxima = max(0, edad - 18)
        antiguedad = random.randint(0, antiguedad_maxima)
        personas_a_cargo = random.randint(0, 7)
        rol = random.choice(roles)
        minusvalia = random.choice(opciones_tf)
        activo = random.choice(opciones_tf)
        id_departamento = random.randint(1, 7)


        writer.writerow([i + 1, f'{nombre} {apellidos}', edad, antiguedad, personas_a_cargo, rol, minusvalia, activo, id_departamento])

print("El archivo 'empleados.csv' ha sido creado exitosamente.")

El archivo 'empleados.csv' ha sido creado exitosamente.


### CSV VACACIONES

In [9]:
num_solicitudes = 100000  # Generaremos algunas solicitudes por empleado
estados = ['aprobada', 'rechazada']
motivos_rechazo_posibles = [
    'Falta de personal en esas fechas',
    'Superposición con otras vacaciones',
    'Solicitud fuera de plazo',
    'Motivo no justificado'
]
num_empleados_generados = 150  # Usamos el mismo número de empleados

with open('../data/vacaciones.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(['id_empleado', 'vacaciones_inicio', 'vacaciones_fin', 'fecha_solicitud', 'estado_solicitud', 'motivo_rechazo'])

    for _ in range(num_solicitudes):
        id_empleado = random.randint(1, num_empleados_generados)
        fecha_solicitud_date = fake.date_between(start_date='-1y', end_date='today')
        fecha_inicio_date = fake.date_between(start_date=fecha_solicitud_date, end_date='+6mo')
        delta = timedelta(days=random.randint(1, 30))
        fecha_fin_date = fecha_inicio_date + delta
        estado_solicitud = random.choice(estados)
        motivo_rechazo = random.choice(motivos_rechazo_posibles) if estado_solicitud == 'rechazada' else ''

        fecha_solicitud = fecha_solicitud_date.strftime('%Y-%m-%d')
        fecha_inicio = fecha_inicio_date.strftime('%Y-%m-%d')
        fecha_fin = fecha_fin_date.strftime('%Y-%m-%d')

        fila = [id_empleado, fecha_inicio, fecha_fin, fecha_solicitud, estado_solicitud, motivo_rechazo]
        writer.writerow(fila)

print("El archivo 'vacaciones.csv' ha sido creado exitosamente.")

El archivo 'vacaciones.csv' ha sido creado exitosamente.


### CSV DEPARTAMENTOS

In [10]:
# Diccionario para almacenar el número de empleados por departamento
empleados_por_departamento = {i: 0 for i in range(1, 7 + 1)}

# Leer el archivo de empleados para contar los empleados por departamento
try:
    with open('../data/empleados.csv', 'r', encoding='utf-8') as archivo_empleados:
        reader_empleados = csv.DictReader(archivo_empleados)
        for row in reader_empleados:
            try:
                id_departamento = int(row['id_departamento'])
                if 1 <= id_departamento <= 7:
                    empleados_por_departamento[id_departamento] += 1
            except ValueError:
                print(f"Advertencia: No se pudo convertir a entero el id_departamento: {row['id_departamento']}")
            except KeyError:
                print("Advertencia: La columna 'id_departamento' no se encontró en el archivo de empleados.")
except FileNotFoundError:
    print("Error: El archivo 'empleados.csv' no fue encontrado. Asegúrate de que esté en el mismo directorio.")
    exit()

with open('../data/departamentos.csv', 'w', newline='', encoding='utf-8') as archivo_departamentos:
    writer_departamentos = csv.writer(archivo_departamentos)
    writer_departamentos.writerow(['id_departamento', 'nombre_departamento', 'porcentaje_maximo_inactivos', 'numero_total_empleados'])

    nombres_departamentos = [
        "Recursos Humanos",
        "Ventas",
        "Marketing",
        "Desarrollo de Producto",
        "Atención al Cliente",
        "Finanzas",
        "Logística"
    ]

    for i in range(1, 7 + 1):
        nombre_departamento = nombres_departamentos[i - 1]
        porcentaje_maximo_inactivos = round(random.uniform(0.1, 0.3), 2) *100
        numero_total_empleados = empleados_por_departamento[i]
        writer_departamentos.writerow([i, nombre_departamento, int(porcentaje_maximo_inactivos), int(numero_total_empleados)])

print("El archivo 'departamentos.csv' ha sido creado exitosamente.")

El archivo 'departamentos.csv' ha sido creado exitosamente.
